# Plan
- step01(PASS) : deepseekocr로 table을 제외한 이미지 구역을 추출
- step02 : 이미지 구역에서 심볼을 기준으로 크롭하고 다음 단계로(sketch_detection_sysmbol_v1,v2)
- step03 :각 크롭한 이미지를 ocr로 텍스트 추출하기 : deepseekocr -> **다른 OCR**(tesseract, paddle)

### setting

In [ ]:
import cv2, os
from symbol_region_splitter import match_symbol, split_and_extract_regions
from utils import visualize_detections, show_image_highres
import pytesseract

### 테스트할 도면과 저장 위치 지정
image_file = '../data/hard_task_test01_v1.PNG'
output_path = './result'
symbol_path = '../data/symbol_mini.PNG'

### step01(방법 고안 필요)

### step02

In [ ]:
# 이미지 로드
drawing = cv2.imread(image_file)
if drawing is None:
    raise FileNotFoundError(f"도면을 찾을 수 없습니다: {image_file}")
drawing_gray = cv2.cvtColor(drawing, cv2.COLOR_BGR2GRAY)

symbol_img = cv2.imread(symbol_path)
if symbol_img is None:
    raise FileNotFoundError(f"심볼 이미지를 찾을 수 없습니다: {symbol_path}")
symbol_gray = cv2.cvtColor(symbol_img, cv2.COLOR_BGR2GRAY)
symbol_name = os.path.splitext(os.path.basename(symbol_path))[0]


# 심볼 매칭 & 중복 제거 & 시각화
filtered_boxes = match_symbol(drawing_gray, symbol_gray, threshold=0.75)
visualize_detections(drawing, filtered_boxes, symbol_name)

# 중심 좌표 계산
centers_x = [(x1 + x2)//2 for (x1, y1, x2, y2) in filtered_boxes]
centers_y = [(y1 + y2)//2 for (x1, y1, x2, y2) in filtered_boxes]

# 영역 분할 & (off)확대 ROI 시각화
regions = split_and_extract_regions(drawing, centers_x, centers_y, fallback=20, zoom_factor=2.0)

# ROI 저장 및 OCR 수행
ocr_results = {}

for region_id, roi in regions:
    # ROI 시각화
    show_image_highres(roi, f"Region {region_id}", zoom=1.0, dpi=150)

    # OCR 수행
    roi_folder = os.path.join(output_path, f'region_{region_id}')
    os.makedirs(roi_folder, exist_ok=True)
    roi_file = os.path.join(roi_folder, f'roi_{region_id}.png')
    cv2.imwrite(roi_file, roi)

    res = model.infer(
        tokenizer,
        prompt=prompt,
        image_file=roi_file,
        output_path=roi_folder,
        base_size=1024,
        image_size=640,
        crop_mode=True,
        save_results=True,
        test_compress=True
    )
    ocr_results[region_id] = res